# Demonstrate functions dealing with em follow-up

In [9]:
import importlib
import photometry_utils
import new_events
importlib.reload(photometry_utils)
importlib.reload(new_events)

<module 'new_events' from '/home/kira/mypy/repos/BBHBot/emfollowup/new_events.py'>

In [10]:
from new_events import Gracedb
params = Gracedb().get_new_events()

120 significant superevents in O4b
https://gracedb.ligo.org/api/superevents/S250108ha/files/none
https://gracedb.ligo.org/api/superevents/S241126dm/files/none
https://gracedb.ligo.org/api/superevents/S241104a/files/none
https://gracedb.ligo.org/api/superevents/S240624cd/files/none
https://gracedb.ligo.org/api/superevents/S240423br/files/none
https://gracedb.ligo.org/api/superevents/S240420aw/files/none
0 events


In [ ]:
from new_events import Fritz

eventid, dateid, a90, far, mass = params[0], params[12], params[16], params[13], params[22]
trigger_status = Fritz(eventid, dateid, a90, far, mass).get_trigger_status()

In [7]:
# check photometry status

from photometry_utils import PhotometryStatus
PhotometryStatus('O4b').show_status()

76 / 98 events in O4b are not priority


,ID,Date last zfps,Status,Trigger
0,S250118dp,2025-02-10 17:36:22.083,update ZFPS,
1,S250119ag,2025-02-10 17:33:12.423,update ZFPS,
2,S250109f,2025-01-24 21:21:42.071,update ZFPS,
3,S250108eo,2025-01-24 21:18:05.241,update ZFPS,
4,S241210cw,2025-01-24 23:32:19.070,update ZFPS,
5,S241129aa,2025-01-24 23:35:29.537,update ZFPS,
6,S241130n,2025-01-24 23:38:29.530,update ZFPS,
7,S241114y,2025-01-24 21:04:56.625,update ZFPS,True
8,S241009em,NA,no AGN,True
9,S241006k,2025-01-14 13:42:29.916,update ZFPS,True


In [6]:
import importlib
import photometry_utils
importlib.reload(photometry_utils)

<module 'photometry_utils' from '/home/kira/mypy/repos/BBHBot/emfollowup/photometry_utils.py'>